In [1]:
#os_key Setting

from dotenv import load_dotenv
import os
# .env 파일 로드(api key load) 
#GOOGLE_API_KEY 
#LANGCHAIN_API_KEY 
#TAVILY_API_KEY 
#HUGGINGFACEHUB_API_TOKEN 
#COHERE_API_KEY 

load_dotenv()

True

In [2]:
#langsmith Setting
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com" 
os.environ["LANGCHAIN_PROJECT"] = "langgraph_tool_binding"            #표시될 PJT명 Setting
#os.environ["LANGCHAIN_API_KEY"] = "lsv2_***"                        *.env로 setting
from langsmith import Client
client = Client()

- @tool 데코레이터 사용
- 가장 간단한 방법은 @tool 데코레이터를 사용하는 것입니다.

In [3]:
#tool을 이용해서 multiply라는 tool을 생성. 함수를 만드는 것과 동일하다
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int: #곰셈 tool
    """두 숫자를 곱합니다."""
    return a * b

In [4]:
#tool을 이용해서 multiply라는 tool을 생성. 함수를 만드는 것과 동일하다
import requests
from bs4 import BeautifulSoup
import re

@tool
def naver_news_crawl(news_url: str) -> str:   #네이버 뉴스 크롤링링
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="newsct_article").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"

In [10]:
url = 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'
response = requests.get(url)
# 요청이 성공했는지 확인
if response.status_code == 200:
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.find("h2", id="title_area").get_text()
    cleaned_title = re.sub(r"\n{2,}", "\n", title)
    content = soup.find("div", id="newsct_article").get_text()
    cleaned_content = re.sub(r"\n{2,}", "\n", content)

In [9]:
print(cleaned_title)

[미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표


In [11]:
print(cleaned_content)


			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국채지수(WGBI) 에 편입될 지 관심. 주식의 경우 지난 2009년부터 선진국 지수에 편입돼 있는 한국 증시에 대해 공매도 제한 등을 이유로 관찰 대상국으로 지정할지 관심. 지정되더라도 검토 기간이 있어 즉각 제외되지는 않음.



In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash-exp', temperature= 0)

tools = [multiply, naver_news_crawl]
llm_with_tools = llm.bind_tools(tools)

result = llm_with_tools.invoke("7과 8을 곱하면 얼마인가요?")
print(result)


Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 7.0, "b": 8.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-b652a17e-4508-4b43-a210-e372d504ac69-0' tool_calls=[{'name': 'multiply', 'args': {'a': 7.0, 'b': 8.0}, 'id': '2b5305eb-5eb5-4608-a41c-8c019c2ac9ba', 'type': 'tool_call'}] usage_metadata={'input_tokens': 122, 'output_tokens': 3, 'total_tokens': 125}


In [14]:
result = llm_with_tools.invoke("뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065").tool_calls
print(result)


[{'name': 'naver_news_crawl', 'args': {'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}, 'id': 'f0a4131e-17f1-4ddb-a6f6-4fa27e6c6775', 'type': 'tool_call'}]


In [15]:
result2 = llm_with_tools.invoke("7과 8을 곱하면 얼마인가요?").tool_calls
print(result2)

[{'name': 'multiply', 'args': {'b': 8.0, 'a': 7.0}, 'id': 'f39874e9-bae4-49bb-af38-c7d4288693b6', 'type': 'tool_call'}]


In [16]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# 도구 바인딩 + 도구 파서
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 결과
tool_call_results = chain.invoke("7과 8을 곱하면 얼마인가요?")

In [17]:
tool_call_results

[{'args': {'b': 8.0, 'a': 7.0}, 'type': 'multiply'}]

- create_tool_calling_agent 사용하기
- AgentExecutor 사용하기

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Agent프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash-exp', temperature= 0)

In [19]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 이전에 정의한 도구 사용
tools = [multiply, naver_news_crawl]

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성(verbose=False)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)


Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [20]:
# AgentExecutor 생성(verbose=True)
agent_executor2 = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [21]:
# Agent 실행
result = agent_executor.invoke({"input": "7과 8을 곱하면 얼마인가요?"})

# 결과 확인
print(result["output"])


7 곱하기 8은 56입니다.



In [22]:
# Agent 실행
result = agent_executor2.invoke({"input": "7과 8을 곱하면 얼마인가요?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 7.0, 'b': 8.0}`


567 곱하기 8은 56입니다.


> Finished chain.
7 곱하기 8은 56입니다.



In [23]:
# Agent 실행
result = agent_executor.invoke({"input":  "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"})

# 결과 확인
print(result["output"])

[미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.
▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.
▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)
▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.
▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.
▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국채지수(WGBI) 에 편입될 지 관심. 주식의 경우 지난 2009년부터 선진국 지수에 편입돼 있는 한국 증시에 대해 공매도 제한 등을 이유로 관찰 대상국으로 지정할지 관심. 지정되더라도 검토 기간이 있어 즉각 제외되지는 않음.



In [24]:
# Agent 실행
result = agent_executor2.invoke({"input":  "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/article/011/0004430917"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/article/011/0004430917'}`


트럼프 취임 전 협상 마무리… TSMC 독주에 파운드리 보조금은 줄어

■ 美반도체 보조금 최종계약삼성 47억弗···투자규모 16% ↓하이닉스 4.5억弗 오히려 증가메모리업계 대부분 예상치 유지
삼성전자의 미국 테일러 공장 전경. 사진제공=삼성전자[서울경제] 삼성전자와 SK하이닉스가 미국 행정부와 반도체 투자에 관한 보조금 계약을 확정 지었다. 트럼프 정부가 출범하기 한 달 전에 계약이 마무리되면서 일단 투자 불확실성은 최소화할 수 있게 됐다. 삼성·인텔 등 현지에 파운드리(반도체 위탁 생산) 설비를 짓고 있는 회사들의 보조금이 당초 예상보다 줄어든 반면 SK하이닉스·마이크론 등 메모리 업체들의 보조금은 예상치가 최초 계획대로 유지돼 반도체 제조와 관련한 양대 사업의 희비가 엇갈렸다는 평가도 나온다.20일(현지 시간) 미 상무부는 텍사스주 테일러시에 파운드리 공장을 짓고 있는 삼성전자에 최대 47억 4500만 달러(약 6조 9000억 원)의 보조금을 직접 지급한다고 발표했다. 이는 삼성전자가 바이든 행정부와 4월에 체결한 예비거래각서(PMT)에서 명시한 보조금 64억 달러(약 9조 3000억 원)보다 26%가량 줄어든 수치다.삼성의 투자금 규모도 달라졌다. 미국의 보조금은 기본적으로 현지에 공장을 짓도록 지원하는 자금이기 때문에 투자 액수에 비례해 보조금 액수도 달라지게 된다. 상무부에 따르면 삼성이 최종 확정한 대미 투자는 370억 달러(약 53조 6000억 원)로 4월 PMT 서명 때 발표한 440억 달러(약 63조 8000억 원)보다 16% 감소했다. 결론적으로 보면 투자금 감액 규모(16%포인트)보다 보조금 감액 규모(26%포인트)가 더 커진 셈이다. 미 상무부 대변인은 보조금 감액 배경에 